# Preprocessing and feature engineering

Here in this notebook we will do some simple preprocessing and feature engineering.
* onehot encoding for categorical features.
* numerical aggregation for other numeric columns.
* some important feature engineering

Important features:
 * age
 * Qualification 
 * experience
 * Dependents
 * Income
 * Type of employment
 * Credit 
 * Payment history
 * Down Payment
 * Annuity
 * Distance

https://www.makaan.com/iq/finance-insurance-saving/9-factors-that-influence-your-prospects-of-getting-a-home-loan

https://homeguides.sfgate.com/ten-important-factors-consider-applying-mortgage-94373.html

https://www.businesstoday.in/current/economy-politics/10-factors-banks-consider-before-approving-your-home-loan/story/251649.html

In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
import os 
os.chdir('drive/My Drive/Home_Credit_Default_Risk')

In [0]:
import pandas as pd
import numpy as np


In [0]:
def onehotenc(df):
  original = list(df.columns)
  cat_col =[]
  for col in df:
    if df[col].dtype == 'object':
      cat_col.append(col)
  df = pd.get_dummies(df,columns=cat_col,dummy_na = True)
  new_col = [c for c in df.columns if c not in original]
  return df,new_col

## application_train_test

In [0]:
train = pd.read_csv('application_train.csv')
test_df = pd.read_csv('application_test.csv')

from sklearn.preprocessing import Imputer
imputer = Imputer(strategy = 'median')

train = imputer.fit_transform(train)
test = imputer.transform(test)
df = train.append(test_df).reset_index()


In [0]:
df.shape,train.shape,test_df.shape

((356255, 123), (307511, 122), (48744, 121))

In [0]:
df.dtypes.value_counts()

float64    66
int64      41
object     16
dtype: int64

Application data.
* categorical features - 16
* numrecal features - 107

In [0]:
df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE             2
CODE_GENDER                    3
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
NAME_TYPE_SUITE                7
NAME_INCOME_TYPE               8
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
OCCUPATION_TYPE               18
WEEKDAY_APPR_PROCESS_START     7
ORGANIZATION_TYPE             58
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
WALLSMATERIAL_MODE             7
EMERGENCYSTATE_MODE            2
dtype: int64

In [0]:
df['CODE_GENDER'].value_counts()

F      202448
M      105059
XNA         4
Name: CODE_GENDER, dtype: int64

Remove data points with `CODE_GENDER = 'XNA'`.

In [0]:
## remove rows with CODE_GENDER = 'XNA'
df = df[df['CODE_GENDER']!='XNA']
df['CODE_GENDER'].value_counts()

F    235126
M    121125
Name: CODE_GENDER, dtype: int64

Zero-one encoding for binary features.

In [0]:
for bin_feat in ['NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','EMERGENCYSTATE_MODE']:
  df[bin_feat],_ = pd.factorize(df[bin_feat])

onehot encoding for categorical features.

In [0]:
df,_ = onehotenc(df)

In [2]:
# shape after encoding.
df.shape

(356255, 252)


Check if days feature have some extreme values, remove if exist.

In [0]:
(df['DAYS_BIRTH']/-365).describe()

count    356255.000000
mean         43.948627
std          11.941929
min          20.104110
25%          34.041096
50%          43.164384
75%          53.906849
max          69.120548
Name: DAYS_BIRTH, dtype: float64

In [0]:
(df['DAYS_EMPLOYED']).describe()

count    356255.000000
mean      64317.231413
std      141705.532576
min      -17912.000000
25%       -2781.000000
50%       -1224.000000
75%        -290.000000
max      365243.000000
Name: DAYS_EMPLOYED, dtype: float64

In [0]:
df['DAYS_EMPLOYED'].replace({365243.000000:np.nan},inplace=True)

In [0]:
df['DAYS_REGISTRATION'].describe()

count    356255.000000
mean      -4983.593527
std        3526.968986
min      -24672.000000
25%       -7477.000000
50%       -4502.000000
75%       -1995.000000
max           0.000000
Name: DAYS_REGISTRATION, dtype: float64

In [0]:
df['DAYS_ID_PUBLISH'].describe()

count    356255.000000
mean      -3002.071163
std        1517.901735
min       -7197.000000
25%       -4318.000000
50%       -3252.000000
75%       -1717.000000
max           0.000000
Name: DAYS_ID_PUBLISH, dtype: float64

How age and employment age correlation  to target?

In [0]:
abs(df['DAYS_BIRTH']).corr(df['TARGET'])

-0.07823930830982694

In [0]:
abs(df['DAYS_EMPLOYED']).corr(df['TARGET'])

-0.07495845742543872

As person get older and more experienced he/she tend to pay their loan on time

In [0]:
correlation = df.corr()['TARGET'].sort_values()

print('Most positive corr: ',correlation.tail(10))
print('Most negative corr: ',correlation.head(10))

Most positive corr:  REGION_RATING_CLIENT_W_CITY       0.060893
DAYS_EMPLOYED                     0.074958
DAYS_BIRTH                        0.078239
TARGET                            1.000000
NAME_EDUCATION_TYPE_nan                NaN
NAME_FAMILY_STATUS_nan                 NaN
NAME_HOUSING_TYPE_nan                  NaN
NAME_INCOME_TYPE_nan                   NaN
ORGANIZATION_TYPE_nan                  NaN
WEEKDAY_APPR_PROCESS_START_nan         NaN
Name: TARGET, dtype: float64
Most negative corr:  EXT_SOURCE_3                           -0.178919
EXT_SOURCE_2                           -0.160472
EXT_SOURCE_1                           -0.155317
NAME_EDUCATION_TYPE_Higher education   -0.056593
CODE_GENDER                            -0.054718
NAME_INCOME_TYPE_Pensioner             -0.046209
ORGANIZATION_TYPE_XNA                  -0.045987
FLOORSMAX_AVG                          -0.044003
FLOORSMAX_MEDI                         -0.043768
FLOORSMAX_MODE                         -0.043226
Name: TAR

Design some new features

In [0]:
## percentage of employment
df['emp_per'] = df['DAYS_EMPLOYED']/df['DAYS_BIRTH']
## percentage of credit
df['income_credit_per'] = df['AMT_INCOME_TOTAL']/df['AMT_CREDIT']
## income per person
df['income_per_person'] = df['AMT_INCOME_TOTAL']/df['CNT_FAM_MEMBERS']
## percentage of annuity to income
df['annuity_income_per'] = df['AMT_ANNUITY']/df['AMT_INCOME_TOTAL']
## payment rate based on annuity and credit
df['payment_rate'] = df['AMT_ANNUITY']/df['AMT_CREDIT']


In [0]:
correlation = df.corr()['TARGET'].sort_values()

print('Most positive corr: ',correlation.tail(10))
print('Most negative corr: ',correlation.head(10))

Most positive corr:  REGION_RATING_CLIENT_W_CITY       0.060893
DAYS_EMPLOYED                     0.074958
DAYS_BIRTH                        0.078239
TARGET                            1.000000
NAME_EDUCATION_TYPE_nan                NaN
NAME_FAMILY_STATUS_nan                 NaN
NAME_HOUSING_TYPE_nan                  NaN
NAME_INCOME_TYPE_nan                   NaN
ORGANIZATION_TYPE_nan                  NaN
WEEKDAY_APPR_PROCESS_START_nan         NaN
Name: TARGET, dtype: float64
Most negative corr:  EXT_SOURCE_3                           -0.178919
EXT_SOURCE_2                           -0.160472
EXT_SOURCE_1                           -0.155317
emp_per                                -0.067955
NAME_EDUCATION_TYPE_Higher education   -0.056593
CODE_GENDER                            -0.054718
NAME_INCOME_TYPE_Pensioner             -0.046209
ORGANIZATION_TYPE_XNA                  -0.045987
FLOORSMAX_AVG                          -0.044003
FLOORSMAX_MEDI                         -0.043768
Name: TAR

In [0]:
df.shape

(356255, 257)

In [0]:
import gc
del test_df
gc.collect()

26

## bureau_and_balance
Apply onehot encoding to categorical features and numerical aggregation for numerical features

In [0]:
bureau = pd.read_csv('bureau.csv')
bureau_balance = pd.read_csv('bureau_balance.csv')

In [0]:
bureau.dtypes.value_counts()

float64    8
int64      6
object     3
dtype: int64

bureau
* categorical features -3
* numerical features - 14

In [0]:
bureau_balance.dtypes.value_counts()

int64     2
object    1
dtype: int64

bureau_balance
* categorical features -1
* numerical features - 2

In [0]:
bureau.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

CREDIT_ACTIVE       4
CREDIT_CURRENCY     4
CREDIT_TYPE        15
dtype: int64

In [0]:
bureau_balance.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

STATUS    8
dtype: int64

onehot encoding

In [0]:
bureau,b_cat = onehotenc(bureau)
bureau_balance,bb_cat = onehotenc(bureau_balance)

In [0]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_ACTIVE_nan,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_CURRENCY_nan,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_TYPE_nan
0,215354,5714462,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,-131,NaN,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,215354,5714463,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,-20,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,215354,5714464,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,-16,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,215354,5714465,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,-16,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,215354,5714466,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,-21,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS_0,STATUS_1,STATUS_2,STATUS_3,STATUS_4,STATUS_5,STATUS_C,STATUS_X,STATUS_nan
0,5715448,0,0,0,0,0,0,0,1,0,0
1,5715448,-1,0,0,0,0,0,0,1,0,0
2,5715448,-2,0,0,0,0,0,0,1,0,0
3,5715448,-3,0,0,0,0,0,0,1,0,0
4,5715448,-4,0,0,0,0,0,0,1,0,0


In [0]:
# numerical aggregation for bureau_balance
bb_aggragation ={'MONTHS_BALANCE':['min','max','size']}
for col in bb_cat:
  bb_aggragation[col] =['mean']

bb_agg = bureau_balance.groupby('SK_ID_BUREAU').agg(bb_aggragation)
bb_agg.columns = pd.Index([e[0]+'_'+e[1] for e in bb_agg.columns.tolist()])
bureau = bureau.join(bb_agg,how='left',on='SK_ID_BUREAU')
bureau.drop(['SK_ID_BUREAU'], axis=1, inplace= True)
del bureau_balance,bb_agg
gc.collect()

0

In [0]:
bureau.select_dtypes(['float64','int64']).apply(pd.Series.nunique, axis = 0)

SK_ID_CURR                305811
DAYS_CREDIT                 2923
CREDIT_DAY_OVERDUE           942
DAYS_CREDIT_ENDDATE        14096
DAYS_ENDDATE_FACT           2917
AMT_CREDIT_MAX_OVERDUE     68251
CNT_CREDIT_PROLONG            10
AMT_CREDIT_SUM            236708
AMT_CREDIT_SUM_DEBT       226537
AMT_CREDIT_SUM_LIMIT       51726
AMT_CREDIT_SUM_OVERDUE      1616
DAYS_CREDIT_UPDATE          2982
AMT_ANNUITY                40321
MONTHS_BALANCE_min            97
MONTHS_BALANCE_max            96
MONTHS_BALANCE_size           97
STATUS_0_mean               2771
STATUS_1_mean               1140
STATUS_2_mean                403
STATUS_3_mean                240
STATUS_4_mean                193
STATUS_5_mean               1169
STATUS_C_mean               2771
STATUS_X_mean               2779
STATUS_nan_mean                1
dtype: int64

In [0]:
bureau.head()

,SK_ID_CURR,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Bad debt,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_ACTIVE_nan,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_CURRENCY_currency 4,CREDIT_CURRENCY_nan,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Cash loan (non-earmarked),CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Interbank credit,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for purchase of shares (margin lending),CREDIT_TYPE_Loan for the purchase of equipment,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mobile operator loan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Real estate loan,CREDIT_TYPE_Unknown type of loan,CREDIT_TYPE_nan,MONTHS_BALANCE_min,MONTHS_BALANCE_max,MONTHS_BALANCE_size,STATUS_0_mean,STATUS_1_mean,STATUS_2_mean,STATUS_3_mean,STATUS_4_mean,STATUS_5_mean,STATUS_C_mean,STATUS_X_mean,STATUS_nan_mean
0,215354,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,-131,NaN,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,215354,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,-20,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,215354,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,-16,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,215354,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,-16,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,215354,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,-21,NaN,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
num_aggr = {'DAYS_CREDIT':['min','max','mean'],
            'CREDIT_DAY_OVERDUE':['min','max','mean'],
            'DAYS_CREDIT_ENDDATE':['min','max','mean'],
            'DAYS_ENDDATE_FACT':['min','max','mean'],
            'AMT_CREDIT_MAX_OVERDUE':['mean'],
            'CNT_CREDIT_PROLONG':['sum'] ,
            'AMT_CREDIT_SUM':['max','min','mean','sum'],
            'AMT_CREDIT_SUM_DEBT':['max','min','mean','sum'],
            'AMT_CREDIT_SUM_LIMIT':['max','min','mean','sum'],
            'AMT_CREDIT_SUM_OVERDUE':['mean'],
            'DAYS_CREDIT_UPDATE':['mean'],
            'AMT_ANNUITY':['max','mean'],
            'MONTHS_BALANCE_min':['min'] ,
            'MONTHS_BALANCE_max':['max'] ,
            'MONTHS_BALANCE_size':['mean','sum']}
cat_aggr = {}
for cat in b_cat:
  cat_aggr[cat] = ['mean']
for cat in bb_cat:
  cat_aggr[cat+'_mean'] = ['mean']

bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggr,**cat_aggr})
bureau_agg.columns = pd.Index(['buro_'+e[0]+'_'+e[1] for e in bureau_agg.columns.tolist()])



In [0]:
bureau_agg.index

Int64Index([100001, 100002, 100003, 100004, 100005, 100007, 100008, 100009,
            100010, 100011,
            ...
            456242, 456243, 456244, 456246, 456247, 456249, 456250, 456253,
            456254, 456255],
           dtype='int64', name='SK_ID_CURR', length=305811)

In [0]:
bureau.select_dtypes(['uint8']).apply(pd.Series.nunique, axis = 0)

CREDIT_ACTIVE_Active                                        2
CREDIT_ACTIVE_Bad debt                                      2
CREDIT_ACTIVE_Closed                                        2
CREDIT_ACTIVE_Sold                                          2
CREDIT_ACTIVE_nan                                           1
CREDIT_CURRENCY_currency 1                                  2
CREDIT_CURRENCY_currency 2                                  2
CREDIT_CURRENCY_currency 3                                  2
CREDIT_CURRENCY_currency 4                                  2
CREDIT_CURRENCY_nan                                         1
CREDIT_TYPE_Another type of loan                            2
CREDIT_TYPE_Car loan                                        2
CREDIT_TYPE_Cash loan (non-earmarked)                       2
CREDIT_TYPE_Consumer credit                                 2
CREDIT_TYPE_Credit card                                     2
CREDIT_TYPE_Interbank credit                                2
CREDIT_T

There are two main features here the status of the credit(active or closed).

In [0]:
## value  1 means the credit is active 
active_credit = bureau[bureau['CREDIT_ACTIVE_Active']==1]
active_agg = active_credit.groupby('SK_ID_CURR').agg(num_aggr)
active_agg.columns = pd.Index(['active_'+e[0]+'_'+e[1] for e in active_agg.columns.tolist()])
bureau_agg = bureau_agg.join(active_agg,how='left',on='SK_ID_CURR')
del active_credit,active_agg
gc.collect()


0

In [0]:
## value  1 means the credit is closed 
closed_credit = bureau[bureau['CREDIT_ACTIVE_Closed']==1]
closed_agg = closed_credit.groupby('SK_ID_CURR').agg(num_aggr)
closed_agg.columns = pd.Index(['closed_'+e[0]+'_'+e[1] for e in closed_agg.columns.tolist()])
bureau_agg = bureau_agg.join(closed_agg,how='left',on='SK_ID_CURR')


del bureau,closed_agg,closed_credit
gc.collect()

0

In [0]:
bureau_agg.head()

,buro_DAYS_CREDIT_min,buro_DAYS_CREDIT_max,buro_DAYS_CREDIT_mean,buro_CREDIT_DAY_OVERDUE_min,buro_CREDIT_DAY_OVERDUE_max,buro_CREDIT_DAY_OVERDUE_mean,buro_DAYS_CREDIT_ENDDATE_min,buro_DAYS_CREDIT_ENDDATE_max,buro_DAYS_CREDIT_ENDDATE_mean,buro_DAYS_ENDDATE_FACT_min,buro_DAYS_ENDDATE_FACT_max,buro_DAYS_ENDDATE_FACT_mean,buro_AMT_CREDIT_MAX_OVERDUE_mean,buro_CNT_CREDIT_PROLONG_sum,buro_AMT_CREDIT_SUM_max,buro_AMT_CREDIT_SUM_min,buro_AMT_CREDIT_SUM_mean,buro_AMT_CREDIT_SUM_sum,buro_AMT_CREDIT_SUM_DEBT_max,buro_AMT_CREDIT_SUM_DEBT_min,buro_AMT_CREDIT_SUM_DEBT_mean,buro_AMT_CREDIT_SUM_DEBT_sum,buro_AMT_CREDIT_SUM_LIMIT_max,buro_AMT_CREDIT_SUM_LIMIT_min,buro_AMT_CREDIT_SUM_LIMIT_mean,buro_AMT_CREDIT_SUM_LIMIT_sum,buro_AMT_CREDIT_SUM_OVERDUE_mean,buro_DAYS_CREDIT_UPDATE_mean,buro_AMT_ANNUITY_max,buro_AMT_ANNUITY_mean,buro_MONTHS_BALANCE_min_min,buro_MONTHS_BALANCE_max_max,buro_MONTHS_BALANCE_size_mean,buro_MONTHS_BALANCE_size_sum,buro_CREDIT_ACTIVE_Active_mean,buro_CREDIT_ACTIVE_Bad debt_mean,buro_CREDIT_ACTIVE_Closed_mean,buro_CREDIT_ACTIVE_Sold_mean,buro_CREDIT_ACTIVE_nan_mean,buro_CREDIT_CURRENCY_currency 1_mean,...,active_AMT_ANNUITY_max,active_AMT_ANNUITY_mean,active_MONTHS_BALANCE_min_min,active_MONTHS_BALANCE_max_max,active_MONTHS_BALANCE_size_mean,active_MONTHS_BALANCE_size_sum,closed_DAYS_CREDIT_min,closed_DAYS_CREDIT_max,closed_DAYS_CREDIT_mean,closed_CREDIT_DAY_OVERDUE_min,closed_CREDIT_DAY_OVERDUE_max,closed_CREDIT_DAY_OVERDUE_mean,closed_DAYS_CREDIT_ENDDATE_min,closed_DAYS_CREDIT_ENDDATE_max,closed_DAYS_CREDIT_ENDDATE_mean,closed_DAYS_ENDDATE_FACT_min,closed_DAYS_ENDDATE_FACT_max,closed_DAYS_ENDDATE_FACT_mean,closed_AMT_CREDIT_MAX_OVERDUE_mean,closed_CNT_CREDIT_PROLONG_sum,closed_AMT_CREDIT_SUM_max,closed_AMT_CREDIT_SUM_min,closed_AMT_CREDIT_SUM_mean,closed_AMT_CREDIT_SUM_sum,closed_AMT_CREDIT_SUM_DEBT_max,closed_AMT_CREDIT_SUM_DEBT_min,closed_AMT_CREDIT_SUM_DEBT_mean,closed_AMT_CREDIT_SUM_DEBT_sum,closed_AMT_CREDIT_SUM_LIMIT_max,closed_AMT_CREDIT_SUM_LIMIT_min,closed_AMT_CREDIT_SUM_LIMIT_mean,closed_AMT_CREDIT_SUM_LIMIT_sum,closed_AMT_CREDIT_SUM_OVERDUE_mean,closed_DAYS_CREDIT_UPDATE_mean,closed_AMT_ANNUITY_max,closed_AMT_ANNUITY_mean,closed_MONTHS_BALANCE_min_min,closed_MONTHS_BALANCE_max_max,closed_MONTHS_BALANCE_size_mean,closed_MONTHS_BALANCE_size_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-1572,-49,-735.000000,0,0,0.0,-1329.0,1778.0,82.428571,-1328.0,-544.0,-825.500000,NaN,0,378000.0,85500.0,207623.571429,1453365.000,373239.0,0.0,85240.928571,596686.5,0.000,0.0,0.00000,0.000,0.0,-93.142857,10822.5,3545.357143,-51.0,0.0,24.571429,172.0,0.428571,0.0,0.571429,0.0,0,1.0,...,10822.5,8272.50,-18.0,0.0,10.666667,32.0,-1572.0,-857.0,-1054.250000,0.0,0.0,0.0,-1329.0,-179.0,-628.500000,-1328.0,-544.0,-825.500000,NaN,0.0,279720.0,85500.0,142335.0,569340.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-155.000000,0.0,0.0,-51.0,0.0,35.0,140.0
100002,-1437,-103,-874.000000,0,0,0.0,-1072.0,780.0,-349.000000,-1185.0,-36.0,-697.500000,1681.029,0,450000.0,0.0,108131.945625,865055.565,245781.0,0.0,49156.200000,245781.0,31988.565,0.0,7997.14125,31988.565,0.0,-499.875000,0.0,0.000000,-47.0,0.0,13.750000,110.0,0.250000,0.0,0.750000,0.0,0,1.0,...,0.0,0.00,-34.0,0.0,10.000000,20.0,-1437.0,-476.0,-974.500000,0.0,0.0,0.0,-1072.0,85.0,-574.800000,-1185.0,-36.0,-697.500000,2091.16125,0.0,135000.0,0.0,63844.5,383067.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-661.333333,0.0,0.0,-47.0,0.0,15.0,90.0
100003,-2586,-606,-1400.750000,0,0,0.0,-2434.0,1216.0,-544.500000,-2131.0,-540.0,-1097.333333,0.000,0,810000.0,22248.0,254350.125000,1017400.500,0.0,0.0,0.000000,0.0,810000.000,0.0,202500.00000,810000.000,0.0,-816.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.250000,0.0,0.750000,0.0,0,1.0,...,NaN,NaN,NaN,NaN,NaN,0.0,-2586.0,-775.0,-1665.666667,0.0,0.0,0.0,-2434.0,-420.0,-1131.333333,-2131.0,-540.0,-1097.333333,0.00000,0.0,112500.0,22248.0,69133.5,207400.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1073.666667,NaN,NaN,NaN,NaN,

## previous_applications

In [0]:
prev_appl = pd.read_csv('previous_application.csv')
prev_appl.shape

(1670214, 37)

In [0]:
prev_appl.dtypes.value_counts()

object     16
float64    15
int64       6
dtype: int64

In [0]:
prev_appl.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
prev_appl.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

NAME_CONTRACT_TYPE              4
WEEKDAY_APPR_PROCESS_START      7
FLAG_LAST_APPL_PER_CONTRACT     2
NAME_CASH_LOAN_PURPOSE         25
NAME_CONTRACT_STATUS            4
NAME_PAYMENT_TYPE               4
CODE_REJECT_REASON              9
NAME_TYPE_SUITE                 7
NAME_CLIENT_TYPE                4
NAME_GOODS_CATEGORY            28
NAME_PORTFOLIO                  5
NAME_PRODUCT_TYPE               3
CHANNEL_TYPE                    8
NAME_SELLER_INDUSTRY           11
NAME_YIELD_GROUP                5
PRODUCT_COMBINATION            17
dtype: int64

In [0]:
prev,cat_col = onehotenc(prev_appl)

In [0]:
prev_appl.select_dtypes(['float64','int64']).apply(pd.Series.nunique, axis = 0)

SK_ID_PREV                   1670214
SK_ID_CURR                    338857
AMT_ANNUITY                   357959
AMT_APPLICATION                93885
AMT_CREDIT                     86803
AMT_DOWN_PAYMENT               29278
AMT_GOODS_PRICE                93885
HOUR_APPR_PROCESS_START           24
NFLAG_LAST_APPL_IN_DAY             2
RATE_DOWN_PAYMENT             207033
RATE_INTEREST_PRIMARY            148
RATE_INTEREST_PRIVILEGED          25
DAYS_DECISION                   2922
SELLERPLACE_AREA                2097
CNT_PAYMENT                       49
DAYS_FIRST_DRAWING              2838
DAYS_FIRST_DUE                  2892
DAYS_LAST_DUE_1ST_VERSION       4605
DAYS_LAST_DUE                   2873
DAYS_TERMINATION                2830
NFLAG_INSURED_ON_APPROVAL          2
dtype: int64

In [0]:
prev['DAYS_FIRST_DRAWING'].describe()

count    997149.000000
mean     342209.855039
std       88916.115834
min       -2922.000000
25%      365243.000000
50%      365243.000000
75%      365243.000000
max      365243.000000
Name: DAYS_FIRST_DRAWING, dtype: float64

In [0]:
prev['DAYS_FIRST_DRAWING'].replace({365243.000000:np.nan},inplace=True)

In [0]:
prev['DAYS_FIRST_DUE'].describe()

count    997149.000000
mean      13826.269337
std       72444.869708
min       -2892.000000
25%       -1628.000000
50%        -831.000000
75%        -411.000000
max      365243.000000
Name: DAYS_FIRST_DUE, dtype: float64

In [0]:
prev['DAYS_FIRST_DUE'].replace({365243.000000:np.nan},inplace=True)

In [0]:
prev['DAYS_LAST_DUE_1ST_VERSION'].describe()

count    997149.000000
mean      33767.774054
std      106857.034789
min       -2801.000000
25%       -1242.000000
50%        -361.000000
75%         129.000000
max      365243.000000
Name: DAYS_LAST_DUE_1ST_VERSION, dtype: float64

In [0]:
prev['DAYS_LAST_DUE_1ST_VERSION'].replace({365243.000000:np.nan},inplace=True)

In [0]:
prev['DAYS_LAST_DUE'].describe()

count    997149.000000
mean      76582.403064
std      149647.415123
min       -2889.000000
25%       -1314.000000
50%        -537.000000
75%         -74.000000
max      365243.000000
Name: DAYS_LAST_DUE, dtype: float64

In [0]:
prev['DAYS_LAST_DUE'].replace({365243.000000:np.nan},inplace=True)

In [0]:
prev['DAYS_TERMINATION'].describe()

count    997149.000000
mean      81992.343838
std      153303.516729
min       -2874.000000
25%       -1270.000000
50%        -499.000000
75%         -44.000000
max      365243.000000
Name: DAYS_TERMINATION, dtype: float64

In [0]:
prev['DAYS_TERMINATION'].replace({365243.000000:np.nan},inplace=True)

In [0]:
prev['DAYS_DECISION'].describe()

count    1.670214e+06
mean    -8.806797e+02
std      7.790997e+02
min     -2.922000e+03
25%     -1.300000e+03
50%     -5.810000e+02
75%     -2.800000e+02
max     -1.000000e+00
Name: DAYS_DECISION, dtype: float64

In [0]:
num_aggr = { 'AMT_ANNUITY': ['min', 'max', 'mean'],
            'AMT_APPLICATION': ['min', 'max', 'mean'],
            'AMT_CREDIT': ['min', 'max', 'mean'],
            'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
            'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
            'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
            'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
            'RATE_INTEREST_PRIMARY': ['min', 'max', 'mean'],
            'RATE_INTEREST_PRIVILEGED': ['min', 'max', 'mean'],
            'DAYS_DECISION': ['min', 'max', 'mean'],
            'CNT_PAYMENT': ['min', 'max', 'mean'] }
cat_aggr ={}
for cat in cat_col:
  cat_aggr[cat] = ['mean']
prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggr,**cat_aggr})
prev_agg.columns = pd.Index(['prev_'+e[0]+'_'+e[1] for e in prev_agg.columns.tolist()])


In [0]:
cat_col

['NAME_CONTRACT_TYPE_Cash loans',
 'NAME_CONTRACT_TYPE_Consumer loans',
 'NAME_CONTRACT_TYPE_Revolving loans',
 'NAME_CONTRACT_TYPE_XNA',
 'NAME_CONTRACT_TYPE_nan',
 'WEEKDAY_APPR_PROCESS_START_FRIDAY',
 'WEEKDAY_APPR_PROCESS_START_MONDAY',
 'WEEKDAY_APPR_PROCESS_START_SATURDAY',
 'WEEKDAY_APPR_PROCESS_START_SUNDAY',
 'WEEKDAY_APPR_PROCESS_START_THURSDAY',
 'WEEKDAY_APPR_PROCESS_START_TUESDAY',
 'WEEKDAY_APPR_PROCESS_START_WEDNESDAY',
 'WEEKDAY_APPR_PROCESS_START_nan',
 'FLAG_LAST_APPL_PER_CONTRACT_N',
 'FLAG_LAST_APPL_PER_CONTRACT_Y',
 'FLAG_LAST_APPL_PER_CONTRACT_nan',
 'NAME_CASH_LOAN_PURPOSE_Building a house or an annex',
 'NAME_CASH_LOAN_PURPOSE_Business development',
 'NAME_CASH_LOAN_PURPOSE_Buying a garage',
 'NAME_CASH_LOAN_PURPOSE_Buying a holiday home / land',
 'NAME_CASH_LOAN_PURPOSE_Buying a home',
 'NAME_CASH_LOAN_PURPOSE_Buying a new car',
 'NAME_CASH_LOAN_PURPOSE_Buying a used car',
 'NAME_CASH_LOAN_PURPOSE_Car repairs',
 'NAME_CASH_LOAN_PURPOSE_Education',
 'NAME_CASH_L

Here status of the application is important wether it is approved or refused.

In [0]:
#NAME_CONTRACT_STATUS_Refused
refused = prev[prev['NAME_CONTRACT_STATUS_Refused']==1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggr)
refused_agg.columns = pd.Index(['refused_'+e[0]+'_'+e[1] for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg,how='left',on='SK_ID_CURR')
del refused,refused_agg
gc.collect()

#NAME_CONTRACT_STATUS_Approved
approved = prev[prev['NAME_CONTRACT_STATUS_Approved']==1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggr)
approved_agg.columns = pd.Index(['approved_'+e[0]+'_'+e[1] for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg,how='left',on='SK_ID_CURR')
del approved,approved_agg
gc.collect()

0

In [0]:
prev_agg.head()

,prev_AMT_ANNUITY_min,prev_AMT_ANNUITY_max,prev_AMT_ANNUITY_mean,prev_AMT_APPLICATION_min,prev_AMT_APPLICATION_max,prev_AMT_APPLICATION_mean,prev_AMT_CREDIT_min,prev_AMT_CREDIT_max,prev_AMT_CREDIT_mean,prev_AMT_DOWN_PAYMENT_min,prev_AMT_DOWN_PAYMENT_max,prev_AMT_DOWN_PAYMENT_mean,prev_AMT_GOODS_PRICE_min,prev_AMT_GOODS_PRICE_max,prev_AMT_GOODS_PRICE_mean,prev_HOUR_APPR_PROCESS_START_min,prev_HOUR_APPR_PROCESS_START_max,prev_HOUR_APPR_PROCESS_START_mean,prev_RATE_DOWN_PAYMENT_min,prev_RATE_DOWN_PAYMENT_max,prev_RATE_DOWN_PAYMENT_mean,prev_RATE_INTEREST_PRIMARY_min,prev_RATE_INTEREST_PRIMARY_max,prev_RATE_INTEREST_PRIMARY_mean,prev_RATE_INTEREST_PRIVILEGED_min,prev_RATE_INTEREST_PRIVILEGED_max,prev_RATE_INTEREST_PRIVILEGED_mean,prev_DAYS_DECISION_min,prev_DAYS_DECISION_max,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_min,prev_CNT_PAYMENT_max,prev_CNT_PAYMENT_mean,prev_NAME_CONTRACT_TYPE_Cash loans_mean,prev_NAME_CONTRACT_TYPE_Consumer loans_mean,prev_NAME_CONTRACT_TYPE_Revolving loans_mean,prev_NAME_CONTRACT_TYPE_XNA_mean,prev_NAME_CONTRACT_TYPE_nan_mean,prev_WEEKDAY_APPR_PROCESS_START_FRIDAY_mean,prev_WEEKDAY_APPR_PROCESS_START_MONDAY_mean,...,refused_RATE_INTEREST_PRIVILEGED_mean,refused_DAYS_DECISION_min,refused_DAYS_DECISION_max,refused_DAYS_DECISION_mean,refused_CNT_PAYMENT_min,refused_CNT_PAYMENT_max,refused_CNT_PAYMENT_mean,approved_AMT_ANNUITY_min,approved_AMT_ANNUITY_max,approved_AMT_ANNUITY_mean,approved_AMT_APPLICATION_min,approved_AMT_APPLICATION_max,approved_AMT_APPLICATION_mean,approved_AMT_CREDIT_min,approved_AMT_CREDIT_max,approved_AMT_CREDIT_mean,approved_AMT_DOWN_PAYMENT_min,approved_AMT_DOWN_PAYMENT_max,approved_AMT_DOWN_PAYMENT_mean,approved_AMT_GOODS_PRICE_min,approved_AMT_GOODS_PRICE_max,approved_AMT_GOODS_PRICE_mean,approved_HOUR_APPR_PROCESS_START_min,approved_HOUR_APPR_PROCESS_START_max,approved_HOUR_APPR_PROCESS_START_mean,approved_RATE_DOWN_PAYMENT_min,approved_RATE_DOWN_PAYMENT_max,approved_RATE_DOWN_PAYMENT_mean,approved_RATE_INTEREST_PRIMARY_min,approved_RATE_INTEREST_PRIMARY_max,approved_RATE_INTEREST_PRIMARY_mean,approved_RATE_INTEREST_PRIVILEGED_min,approved_RATE_INTEREST_PRIVILEGED_max,approved_RATE_INTEREST_PRIVILEGED_mean,approved_DAYS_DECISION_min,approved_DAYS_DECISION_max,approved_DAYS_DECISION_mean,approved_CNT_PAYMENT_min,approved_CNT_PAYMENT_max,approved_CNT_PAYMENT_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,3951.000,3951.000,3951.000,24835.5,24835.5,24835.50,23787.0,23787.0,23787.00,2520.0,2520.0,2520.0,24835.5,24835.5,24835.5,13,13,13.000000,0.104326,0.104326,0.104326,NaN,NaN,NaN,NaN,NaN,NaN,-1740,-1740,-1740.0,8.0,8.0,8.0,0.000000,1.000000,0.0,0.0,0,1.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3951.000,3951.000,3951.000,24835.5,24835.5,24835.5,23787.0,23787.0,23787.0,2520.0,2520.0,2520.0,24835.5,24835.5,24835.5,13.0,13.0,13.000000,0.104326,0.104326,0.104326,NaN,NaN,NaN,NaN,NaN,NaN,-1740.0,-1740.0,-1740.0,8.0,8.0,8.0
100002,9251.775,9251.775,9251.775,179055.0,179055.0,179055.00,179055.0,179055.0,179055.00,0.0,0.0,0.0,179055.0,179055.0,179055.0,9,9,9.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-606,-606,-606.0,24.0,24.0,24.0,0.000000,1.000000,0.0,0.0,0,0.000000,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9251.775,9251.775,9251.775,179055.0,179055.0,179055.0,179055.0,179055.0,179055.0,0.0,0.0,0.0,179055.0,179055.0,179055.0,9.0,9.0,9.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-606.0,-606.0,-606.0,24.0,24.0,24.0
100003,6737.310,98356.995,56553.990,68809.5,900000.0,435436.50,68053.5,1035882.0,484191.00,0.0,6885.0,3442.5,68809.5,900000.0,435436.5,12,17,14.666667,0.000000,0.100061,0.050030,NaN,NaN,NaN,NaN,NaN,NaN,-2341,-746,-1305.0,6.0,12.0,10.0,0.333333,0.666667,0.0,0.0,0,0.333333,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6737.310,98356.995,56553.990,68809.5,900000.0,435436.5,68053.5,1035882.0,484191.0,0.0,6885.0,3442.5,68809.5,900000.0,435436.5,12.0,17.0,14.666667,0.000000,0.100061,0.050030,NaN,NaN,NaN,NaN,NaN,NaN,-2341

## POS_CASH_balance

In [0]:
pos_cash = pd.read_csv('POS_CASH_balance.csv')
pos_cash.shape

(10001358, 8)

In [0]:
pos_cash.dtypes.value_counts()

int64      5
float64    2
object     1
dtype: int64

In [0]:
pos_cash.columns.tolist()

['SK_ID_PREV',
 'SK_ID_CURR',
 'MONTHS_BALANCE',
 'CNT_INSTALMENT',
 'CNT_INSTALMENT_FUTURE',
 'NAME_CONTRACT_STATUS',
 'SK_DPD',
 'SK_DPD_DEF']

In [0]:
pos_cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [0]:
pos,cat_col = onehotenc(pos_cash)

In [0]:
pos_cash.select_dtypes(['object']).apply(pd.Series.nunique,axis=0)

NAME_CONTRACT_STATUS    9
dtype: int64

In [0]:
pos_cash.select_dtypes(['float64','int64']).apply(pd.Series.nunique,axis=0)

SK_ID_PREV               936325
SK_ID_CURR               337252
MONTHS_BALANCE               96
CNT_INSTALMENT               73
CNT_INSTALMENT_FUTURE        79
SK_DPD                     3400
SK_DPD_DEF                 2307
dtype: int64

In [0]:
aggr = {'MONTHS_BALANCE':['min','max','mean'],
        'CNT_INSTALMENT':['sum'],
        'CNT_INSTALMENT_FUTURE':['sum'],
        'SK_DPD':['min','max','mean'],
        'SK_DPD_DEF':['min','max','mean'] }
cat_aggr = {}
for cat in cat_col:
  cat_aggr[cat]=['mean']

pos_aggr = pos.groupby('SK_ID_CURR').agg({**aggr,**cat_aggr})
pos_aggr.columns = pd.Index(['pos_'+e[0]+'_'+e[1] for e in pos_aggr.columns.tolist()])


In [0]:
pos_aggr.shape

(337252, 21)

In [0]:
 pos_aggr.head()

,pos_MONTHS_BALANCE_min,pos_MONTHS_BALANCE_max,pos_MONTHS_BALANCE_mean,pos_CNT_INSTALMENT_sum,pos_CNT_INSTALMENT_FUTURE_sum,pos_SK_DPD_min,pos_SK_DPD_max,pos_SK_DPD_mean,pos_SK_DPD_DEF_min,pos_SK_DPD_DEF_max,pos_SK_DPD_DEF_mean,pos_NAME_CONTRACT_STATUS_Active_mean,pos_NAME_CONTRACT_STATUS_Amortized debt_mean,pos_NAME_CONTRACT_STATUS_Approved_mean,pos_NAME_CONTRACT_STATUS_Canceled_mean,pos_NAME_CONTRACT_STATUS_Completed_mean,pos_NAME_CONTRACT_STATUS_Demand_mean,pos_NAME_CONTRACT_STATUS_Returned to the store_mean,pos_NAME_CONTRACT_STATUS_Signed_mean,pos_NAME_CONTRACT_STATUS_XNA_mean,pos_NAME_CONTRACT_STATUS_nan_mean
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,-96,-53,-72.555556,36.0,13.0,0,7,0.777778,0,7,0.777778,0.777778,0.0,0.0,0.0,0.222222,0.0,0.0,0.000000,0.0,0
100002,-19,-1,-10.000000,456.0,285.0,0,0,0.000000,0,0,0.000000,1.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0
100003,-77,-18,-43.785714,283.0,162.0,0,0,0.000000,0,0,0.000000,0.928571,0.0,0.0,0.0,0.071429,0.0,0.0,0.000000,0.0,0
100004,-27,-24,-25.500000,15.0,9.0,0,0,0.000000,0,0,0.000000,0.750000,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0
100005,-25,-15,-20.000000,117.0,72.0,0,0,0.000000,0,0,0.000000,0.818182,0.0,0.0,0.0,0.090909,0.0,0.0,0.090909,0.0,0


## installments_payments

In [0]:
installments = pd.read_csv('installments_payments.csv')
installments.shape

(13605401, 8)

In [0]:
installments.dtypes.value_counts()

float64    5
int64      3
dtype: int64

In [0]:
installments.select_dtypes(['float64','int64']).apply(pd.Series.nunique,axis=0)

SK_ID_PREV                997752
SK_ID_CURR                339587
NUM_INSTALMENT_VERSION        65
NUM_INSTALMENT_NUMBER        277
DAYS_INSTALMENT             2922
DAYS_ENTRY_PAYMENT          3039
AMT_INSTALMENT            902539
AMT_PAYMENT               944235
dtype: int64

In [0]:
installments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


There are some important factores to be considered as below:
* wether customer pays installment on time, before due date or after due date
* wether customer pays full amount of installment or less then the amount of installment
* percentage of payment to the installment amount.



In [0]:
installments['before_due_payment'] = (installments['DAYS_INSTALMENT']-installments['DAYS_ENTRY_PAYMENT']).apply(lambda x:x if x>0 else 0)
installments['after_due_payment'] = (installments['DAYS_ENTRY_PAYMENT']-installments['DAYS_INSTALMENT']).apply(lambda x:x if x>0 else 0)

installments['paymen_diff'] = installments['AMT_INSTALMENT']-installments['AMT_PAYMENT']
installments['payment_perc'] = installments['AMT_PAYMENT']/installments['DAYS_INSTALMENT']

installments.columns.tolist()

['SK_ID_PREV',
 'SK_ID_CURR',
 'NUM_INSTALMENT_VERSION',
 'NUM_INSTALMENT_NUMBER',
 'DAYS_INSTALMENT',
 'DAYS_ENTRY_PAYMENT',
 'AMT_INSTALMENT',
 'AMT_PAYMENT',
 'before_due_payment',
 'after_due_payment',
 'paymen_diff',
 'payment_perc']

In [0]:
installments['NUM_INSTALMENT_VERSION'].unique()

array([  1.,   0.,   2.,   4.,   3.,   5.,   7.,   8.,   6.,  13.,   9.,
        21.,  22.,  12.,  17.,  18.,  11.,  14.,  34.,  33.,  19.,  16.,
        15.,  10.,  26.,  27.,  20.,  25.,  23.,  24.,  31.,  32.,  28.,
        35.,  29.,  30.,  43.,  39.,  40.,  36.,  41.,  42.,  37.,  38.,
        68.,  44.,  45.,  46., 178.,  52.,  51.,  53.,  54.,  49.,  50.,
        58.,  57.,  55.,  56.,  48.,  47.,  72.,  59.,  73.,  61.])

In [0]:
aggr = {'NUM_INSTALMENT_VERSION':['nunique'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min','max', 'mean', 'sum'],
        'before_due_payment': ['max', 'mean', 'sum'],
        'after_due_payment': ['max', 'mean', 'sum'],
        'paymen_diff': ['max', 'mean', 'sum'],
        'payment_perc': ['max', 'mean', 'sum']}

ins_agg = installments.groupby('SK_ID_CURR').agg(aggr)
ins_agg.columns = pd.Index(['ins_'+e[0]+'_'+e[1] for e in ins_agg.columns.tolist()])

In [0]:
ins_agg.head()

,ins_NUM_INSTALMENT_VERSION_nunique,ins_AMT_INSTALMENT_max,ins_AMT_INSTALMENT_mean,ins_AMT_INSTALMENT_sum,ins_AMT_PAYMENT_min,ins_AMT_PAYMENT_max,ins_AMT_PAYMENT_mean,ins_AMT_PAYMENT_sum,ins_before_due_payment_max,ins_before_due_payment_mean,ins_before_due_payment_sum,ins_after_due_payment_max,ins_after_due_payment_mean,ins_after_due_payment_sum,ins_paymen_diff_max,ins_paymen_diff_mean,ins_paymen_diff_sum,ins_payment_perc_max,ins_payment_perc_mean,ins_payment_perc_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,
100001,2,17397.900,5885.132143,41195.925,3951.000,17397.900,5885.132143,41195.925,36.0,8.857143,62.0,11.0,1.571429,11.0,0.0,0.0,0.0,-1.365586,-3.135199,-21.946390
100002,2,53093.745,11559.247105,219625.695,9251.775,53093.745,11559.247105,219625.695,31.0,20.421053,388.0,0.0,0.000000,0.0,0.0,0.0,0.0,-16.374823,-154.832306,-2941.813813
100003,2,560835.360,64754.586000,1618864.650,6662.970,560835.360,64754.586000,1618864.650,14.0,7.160000,179.0,0.0,0.000000,0.0,0.0,0.0,0.0,-2.916584,-101.981214,-2549.530359
100004,2,10573.965,7096.155000,21288.465,5357.250,10573.965,7096.155000,21288.465,11.0,7.666667,23.0,0.0,0.000000,0.0,0.0,0.0,0.0,-6.833227,-9.514419,-28.543257
100005,2,17656.245,6240.205000,56161.845,4813.200,17656.245,6240.205000,56161.845,37.0,23.666667,213.0,1.0,0.111111,1.0,0.0,0.0,0.0,-6.817564,-11.424013,-102.816119


## credit_card_balance

In [0]:
credit_card = pd.read_csv('credit_card_balance.csv')
credit_card.shape

(3840312, 23)

In [0]:
credit_card.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [0]:
credit_card.dtypes.value_counts()

float64    15
int64       7
object      1
dtype: int64

In [0]:
credit_card,cat_col = onehotenc(credit_card)

In [0]:
credit_card.columns.tolist()

['SK_ID_PREV',
 'SK_ID_CURR',
 'MONTHS_BALANCE',
 'AMT_BALANCE',
 'AMT_CREDIT_LIMIT_ACTUAL',
 'AMT_DRAWINGS_ATM_CURRENT',
 'AMT_DRAWINGS_CURRENT',
 'AMT_DRAWINGS_OTHER_CURRENT',
 'AMT_DRAWINGS_POS_CURRENT',
 'AMT_INST_MIN_REGULARITY',
 'AMT_PAYMENT_CURRENT',
 'AMT_PAYMENT_TOTAL_CURRENT',
 'AMT_RECEIVABLE_PRINCIPAL',
 'AMT_RECIVABLE',
 'AMT_TOTAL_RECEIVABLE',
 'CNT_DRAWINGS_ATM_CURRENT',
 'CNT_DRAWINGS_CURRENT',
 'CNT_DRAWINGS_OTHER_CURRENT',
 'CNT_DRAWINGS_POS_CURRENT',
 'CNT_INSTALMENT_MATURE_CUM',
 'SK_DPD',
 'SK_DPD_DEF',
 'NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed',
 'NAME_CONTRACT_STATUS_nan']

In [0]:
aggr = {'MONTHS_BALANCE':['min','max','mean','sum'],
 'AMT_BALANCE':['min','max','mean','sum'],
 'AMT_CREDIT_LIMIT_ACTUAL':['min','max','mean','sum'],
 'AMT_DRAWINGS_ATM_CURRENT':['min','max','mean','sum'],
 'AMT_DRAWINGS_CURRENT':['min','max','mean','sum'],
 'AMT_DRAWINGS_OTHER_CURRENT':['min','max','mean','sum'],
 'AMT_DRAWINGS_POS_CURRENT':['min','max','mean','sum'],
 'AMT_INST_MIN_REGULARITY':['min','max','mean','sum'],
 'AMT_PAYMENT_CURRENT':['min','max','mean','sum'],
 'AMT_PAYMENT_TOTAL_CURRENT':['min','max','mean','sum'],
 'AMT_RECEIVABLE_PRINCIPAL':['min','max','mean','sum'],
 'AMT_RECIVABLE':['min','max','mean','sum'],
 'AMT_TOTAL_RECEIVABLE':['min','max','mean','sum'],
 'CNT_DRAWINGS_ATM_CURRENT':['sum'],
 'CNT_DRAWINGS_CURRENT':['sum'],
 'CNT_DRAWINGS_OTHER_CURRENT':['sum'],
 'CNT_DRAWINGS_POS_CURRENT':['sum'],
 'CNT_INSTALMENT_MATURE_CUM':['sum'],
 'SK_DPD':['min','max','mean','sum'],
 'SK_DPD_DEF':['min','max','mean','sum']}

cc_agg = credit_card.groupby('SK_ID_CURR').agg(aggr)
cc_agg.columns = pd.Index(['cc_'+e[0]+'_'+e[1] for e in cc_agg.columns.tolist()])


In [0]:
cc_agg.head()

,cc_MONTHS_BALANCE_min,cc_MONTHS_BALANCE_max,cc_MONTHS_BALANCE_mean,cc_MONTHS_BALANCE_sum,cc_AMT_BALANCE_min,cc_AMT_BALANCE_max,cc_AMT_BALANCE_mean,cc_AMT_BALANCE_sum,cc_AMT_CREDIT_LIMIT_ACTUAL_min,cc_AMT_CREDIT_LIMIT_ACTUAL_max,cc_AMT_CREDIT_LIMIT_ACTUAL_mean,cc_AMT_CREDIT_LIMIT_ACTUAL_sum,cc_AMT_DRAWINGS_ATM_CURRENT_min,cc_AMT_DRAWINGS_ATM_CURRENT_max,cc_AMT_DRAWINGS_ATM_CURRENT_mean,cc_AMT_DRAWINGS_ATM_CURRENT_sum,cc_AMT_DRAWINGS_CURRENT_min,cc_AMT_DRAWINGS_CURRENT_max,cc_AMT_DRAWINGS_CURRENT_mean,cc_AMT_DRAWINGS_CURRENT_sum,cc_AMT_DRAWINGS_OTHER_CURRENT_min,cc_AMT_DRAWINGS_OTHER_CURRENT_max,cc_AMT_DRAWINGS_OTHER_CURRENT_mean,cc_AMT_DRAWINGS_OTHER_CURRENT_sum,cc_AMT_DRAWINGS_POS_CURRENT_min,cc_AMT_DRAWINGS_POS_CURRENT_max,cc_AMT_DRAWINGS_POS_CURRENT_mean,cc_AMT_DRAWINGS_POS_CURRENT_sum,cc_AMT_INST_MIN_REGULARITY_min,cc_AMT_INST_MIN_REGULARITY_max,cc_AMT_INST_MIN_REGULARITY_mean,cc_AMT_INST_MIN_REGULARITY_sum,cc_AMT_PAYMENT_CURRENT_min,cc_AMT_PAYMENT_CURRENT_max,cc_AMT_PAYMENT_CURRENT_mean,cc_AMT_PAYMENT_CURRENT_sum,cc_AMT_PAYMENT_TOTAL_CURRENT_min,cc_AMT_PAYMENT_TOTAL_CURRENT_max,cc_AMT_PAYMENT_TOTAL_CURRENT_mean,cc_AMT_PAYMENT_TOTAL_CURRENT_sum,cc_AMT_RECEIVABLE_PRINCIPAL_min,cc_AMT_RECEIVABLE_PRINCIPAL_max,cc_AMT_RECEIVABLE_PRINCIPAL_mean,cc_AMT_RECEIVABLE_PRINCIPAL_sum,cc_AMT_RECIVABLE_min,cc_AMT_RECIVABLE_max,cc_AMT_RECIVABLE_mean,cc_AMT_RECIVABLE_sum,cc_AMT_TOTAL_RECEIVABLE_min,cc_AMT_TOTAL_RECEIVABLE_max,cc_AMT_TOTAL_RECEIVABLE_mean,cc_AMT_TOTAL_RECEIVABLE_sum,cc_CNT_DRAWINGS_ATM_CURRENT_sum,cc_CNT_DRAWINGS_CURRENT_sum,cc_CNT_DRAWINGS_OTHER_CURRENT_sum,cc_CNT_DRAWINGS_POS_CURRENT_sum,cc_CNT_INSTALMENT_MATURE_CUM_sum,cc_SK_DPD_min,cc_SK_DPD_max,cc_SK_DPD_mean,cc_SK_DPD_sum,cc_SK_DPD_DEF_min,cc_SK_DPD_DEF_max,cc_SK_DPD_DEF_mean,cc_SK_DPD_DEF_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100006,-6,-1,-3.5,-21,0.0,0.00,0.000000,0.000,270000,270000,270000.000000,1620000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000,NaN,NaN,NaN,0.00,0.0,0.0,0.000000,0.000,0.0,0.0,0.000000,0.000,0.000,0.00,0.000000,0.000,0.000,0.00,0.000000,0.000,0.0,0,0.0,0.0,0.0,0,0,0.000000,0,0,0,0.000000,0
100011,-75,-2,-38.5,-2849,0.0,189000.00,54482.111149,4031676.225,90000,180000,164189.189189,12150000,0.0,180000.0,2432.432432,180000.0,0.0,180000.0,2432.432432,180000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9000.0,3956.221849,288804.195,0.0,55485.0,4843.064189,358386.75,0.0,55485.0,4520.067568,334485.000,0.0,180000.0,52402.088919,3877754.580,-563.355,189000.00,54433.179122,4028055.255,-563.355,189000.00,54433.179122,4028055.255,4.0,4,0.0,0.0,1881.0,0,0,0.000000,0,0,0,0.000000,0
100013,-96,-1,-48.5,-4656,0.0,161420.22,18159.919219,1743352.245,45000,157500,131718.750000,12645000,0.0,157500.0,6350.000000,571500.0,0.0,157500.0,5953.125000,571500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7875.0,1454.539551,129454.020,0.0,153675.0,7168.346250,688161.24,0.0,153675.0,6817.172344,654448.545,0.0,157500.0,17255.559844,1656533.745,-274.320,161420.22,18101.079844,1737703.665,-274.320,161420.22,18101.079844,1737703.665,23.0,23,0.0,0.0,1666.0,0,1,0.010417,1,0,1,0.010417,1
100021,-18,-2,-10.0,-170,0.0,0.00,0.000000,0.000,675000,675000,675000.000000,11475000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000,NaN,NaN,NaN,0.00,0.0,0.0,0.000000,0.000,0.0,0.0,0.000000,0.000,0.000,0.00,0.000000,0.000,0.000,0.00,0.000000,0.000,0.0,0,0.0,0.0,0.0,0,0,0.000000,0,0,0,0.000000,0
100023,-11,-4,-7.5,-60,0.0,0.00,0.000000,0.000,45000,225000,135000.000000,1080000,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.000,NaN,NaN,NaN,0.00,0.0,0.0,0.000000,0.000,0.0,0.0,0.000000,0.000,0.000,0.00,0.000000,0.000,0.000,0.00,0.000000,0.000,0.0,0,0.0,0.0,0.0,0,0,0.000000,0,0,0,0.000000,0


In [0]:
cc_agg.columns.tolist()

['cc_MONTHS_BALANCE_min',
 'cc_MONTHS_BALANCE_max',
 'cc_MONTHS_BALANCE_mean',
 'cc_MONTHS_BALANCE_sum',
 'cc_AMT_BALANCE_min',
 'cc_AMT_BALANCE_max',
 'cc_AMT_BALANCE_mean',
 'cc_AMT_BALANCE_sum',
 'cc_AMT_CREDIT_LIMIT_ACTUAL_min',
 'cc_AMT_CREDIT_LIMIT_ACTUAL_max',
 'cc_AMT_CREDIT_LIMIT_ACTUAL_mean',
 'cc_AMT_CREDIT_LIMIT_ACTUAL_sum',
 'cc_AMT_DRAWINGS_ATM_CURRENT_min',
 'cc_AMT_DRAWINGS_ATM_CURRENT_max',
 'cc_AMT_DRAWINGS_ATM_CURRENT_mean',
 'cc_AMT_DRAWINGS_ATM_CURRENT_sum',
 'cc_AMT_DRAWINGS_CURRENT_min',
 'cc_AMT_DRAWINGS_CURRENT_max',
 'cc_AMT_DRAWINGS_CURRENT_mean',
 'cc_AMT_DRAWINGS_CURRENT_sum',
 'cc_AMT_DRAWINGS_OTHER_CURRENT_min',
 'cc_AMT_DRAWINGS_OTHER_CURRENT_max',
 'cc_AMT_DRAWINGS_OTHER_CURRENT_mean',
 'cc_AMT_DRAWINGS_OTHER_CURRENT_sum',
 'cc_AMT_DRAWINGS_POS_CURRENT_min',
 'cc_AMT_DRAWINGS_POS_CURRENT_max',
 'cc_AMT_DRAWINGS_POS_CURRENT_mean',
 'cc_AMT_DRAWINGS_POS_CURRENT_sum',
 'cc_AMT_INST_MIN_REGULARITY_min',
 'cc_AMT_INST_MIN_REGULARITY_max',
 'cc_AMT_INST_MIN_R

## combine all the dataframe
Combine and save all new data frame.

In [0]:
#df,bureau_agg,prev_agg,pos_aggr,ins_agg,cc_agg
df = df.join(bureau_agg, how='left', on='SK_ID_CURR')
df = df.join(prev_agg, how='left', on='SK_ID_CURR')
df = df.join(pos_aggr, how='left', on='SK_ID_CURR')
df = df.join(ins_agg, how='left', on='SK_ID_CURR')
df = df.join(cc_agg, how='left', on='SK_ID_CURR')

In [0]:
df.shape

(356251, 758)

In [0]:
df.to_csv('home_features.csv')

In [0]:
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,...,cc_AMT_DRAWINGS_POS_CURRENT_max,cc_AMT_DRAWINGS_POS_CURRENT_mean,cc_AMT_DRAWINGS_POS_CURRENT_sum,cc_AMT_INST_MIN_REGULARITY_min,cc_AMT_INST_MIN_REGULARITY_max,cc_AMT_INST_MIN_REGULARITY_mean,cc_AMT_INST_MIN_REGULARITY_sum,cc_AMT_PAYMENT_CURRENT_min,cc_AMT_PAYMENT_CURRENT_max,cc_AMT_PAYMENT_CURRENT_mean,cc_AMT_PAYMENT_CURRENT_sum,cc_AMT_PAYMENT_TOTAL_CURRENT_min,cc_AMT_PAYMENT_TOTAL_CURRENT_max,cc_AMT_PAYMENT_TOTAL_CURRENT_mean,cc_AMT_PAYMENT_TOTAL_CURRENT_sum,cc_AMT_RECEIVABLE_PRINCIPAL_min,cc_AMT_RECEIVABLE_PRINCIPAL_max,cc_AMT_RECEIVABLE_PRINCIPAL_mean,cc_AMT_RECEIVABLE_PRINCIPAL_sum,cc_AMT_RECIVABLE_min,cc_AMT_RECIVABLE_max,cc_AMT_RECIVABLE_mean,cc_AMT_RECIVABLE_sum,cc_AMT_TOTAL_RECEIVABLE_min,cc_AMT_TOTAL_RECEIVABLE_max,cc_AMT_TOTAL_RECEIVABLE_mean,cc_AMT_TOTAL_RECEIVABLE_sum,cc_CNT_DRAWINGS_ATM_CURRENT_sum,cc_CNT_DRAWINGS_CURRENT_sum,cc_CNT_DRAWINGS_OTHER_CURRENT_sum,cc_CNT_DRAWINGS_POS_CURRENT_sum,cc_CNT_INSTALMENT_MATURE_CUM_sum,cc_SK_DPD_min,cc_SK_DPD_max,cc_SK_DPD_mean,cc_SK_DPD_sum,cc_SK_DPD_DEF_min,cc_SK_DPD_DEF_max,cc_SK_DPD_DEF_mean,cc_SK_DPD_DEF_sum
0,100002,1,0,0,0,0,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637.0,-3648.0,-2120,NaN,1,1,0,1,1,0,1.0,2,2,10,0,0,0,0,0,0,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,0,0,1,0,1,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188.0,-1186.0,-291,NaN,1,1,0,1,1,0,2.0,1,1,11,0,0,0,0,0,0,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,0,1,0,1,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225.0,-4260.0,-2531,26.0,1,1,1,1,1,0,1.0,2,2,9,0,0,0,0,0,0,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,0,0,1,0,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039.0,-9833.0,-2437,NaN,1,1,0,1,0,0,2.0,2,2,17,0,0,0,0,0,0,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,0,0,0,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038.0,-4311.0,-3458,NaN,1,1,0,1,0,0,1.0,2,2,11,0,0,0,0,1,1,NaN,0.322738,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
